# Business logic

### Connect to database

In [3]:
import requests
import mariadb
import sys
import pandas       as pd
import numpy        as np
import datetime     as dt
from scipy              import stats
from pandas.api.types   import is_numeric_dtype



# Connect to MariaDB Platform
try:
    conn = mariadb.connect(
        user="admin_vtx",
        password="Prueba12#",
        host="develop-free-vtx.csgmphwpjwca.us-east-1.rds.amazonaws.com",
        port=3306,
        database="catalogs")
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

In [4]:
users_dict = {
    "cristiano": {
        "instagram": "cristiano",
        "twitter": "Cristiano"
    },
    "messi": {
        "instagram": "leomessi",
        "twitter": "Ieomessiok"
    },
    "lebron": {
        "instagram": "lebron",
        "twitter": "KingJames"
    },
    "nadal": {
        "instagram": "rafaelnadal",
        "twitter": "RafaelNadal"
    },
    "mayweather": {
        "instagram": "floydmayweather",
        "twitter": "FloydMayweather"
    }
}

In [126]:
for username in users_dict.keys():
    ig_username = users_dict[username]["instagram"]
    tw_username = users_dict[username]["twitter"]
    str_to_ex = f"INSERT INTO users (name, instagram_username, twitter_username, last_update) VALUES ('{username}', '{ig_username}', '{tw_username}', now());"
    cur.execute(str_to_ex)
str_to_ex = f"INSERT INTO users (name, instagram_username, last_update) VALUES ('FalcaoGarcia', 'falcao', now());"
cur.execute(str_to_ex)
conn.commit()

Retrieve users

In [7]:
cur.execute("SELECT * FROM users;")

ids = []
names = []
instagram_usernames = []
twitter_usernames = []

for id, name, instagram_username, twitter_username, last_update, gender, date__birth, sport, position, email in  cur: 
    ids.append(id)
    names.append(name)
    instagram_usernames.append(instagram_username)
    twitter_usernames.append(twitter_username)
    print(f"name: {name}, instagram_username: {instagram_username}, twitter_username: {twitter_username}, last_update: {last_update}")

name: Stephany Cruz, instagram_username: estefanycruz90, twitter_username: triplejumperGUA, last_update: 2022-12-05 22:08:53
name: Bernardo Uriel Barrondo, instagram_username: urielbarrondo, twitter_username: nan, last_update: 2022-12-05 22:08:53
name: Maria Fernanda Peinado, instagram_username: marifer7104, twitter_username: nan, last_update: 2022-12-05 22:08:53
name: Danisha Chimillio, instagram_username: dashachimilio05, twitter_username: DarshyCastillo, last_update: 2022-12-05 22:08:54
name: Katya Oliveros, instagram_username: katya_oliveros27, twitter_username: nan, last_update: 2022-12-05 22:08:54
name: Maxwell Alvarez, instagram_username: maxwell.alvarez.98, twitter_username: nan, last_update: 2022-12-05 22:08:54
name: Guillermo Rivas, instagram_username: guillermoflores156, twitter_username: nan, last_update: 2022-12-05 22:08:54
name: Leonardo Oliveros, instagram_username: jeremx347, twitter_username: nan, last_update: 2022-12-05 22:08:54
name: Laura Lainfiesta, instagram_usern

In [23]:
for ig_name in instagram_usernames:
    if ig_name is not None and ig_name != "nan" and ig_name != "":
        print(ig_name)

estefanycruz90
urielbarrondo
marifer7104
dashachimilio05
katya_oliveros27
maxwell.alvarez.98
guillermoflores156
jeremx347
laura_lainfiesta
efra_maza
lestherarevalo
patrickruizj
10wanderley
cristiano
robimvera
leomessi
lebron
rafaelnadal
floydmayweather
falcao
in_aws-lambda3
in_nickname


### Extract data

In [50]:
url_ig = "http://localhost:6000/api/instagram"
url_tw = "http://localhost:6000/api/twitter"

users_data = []

for ix, id in enumerate(ids):
    user_data_dict = {}
    user_data_dict["id"] = id
    user_data_dict["name"] = names[ix]
    user_data_dict["instagram_username"] = instagram_usernames[ix]
    user_data_dict["twitter_username"] = twitter_usernames[ix]
    print(f"Working on {names[ix]}")

    if user_data_dict["instagram_username"] != None:
        
        print("Getting IG data")
        body = {"username": user_data_dict["instagram_username"]}
        r = requests.post(url_ig, json=body)
        response = r.json()
        user_data_dict["instagram_data"] = response

    if user_data_dict["twitter_username"] != None:
        print("Getting TW data")
        body = {"username": user_data_dict["twitter_username"]}
        r = requests.post(url_tw, json=body)
        response = r.json()
        user_data_dict["twitter_data"] = response
    
    users_data.append(user_data_dict)
    print(f"Done with {names[ix]}")
    print("----------------------------------")

Working on cristiano
Getting IG data
Getting TW data
Done with cristiano
----------------------------------
Working on messi
Getting IG data
Getting TW data
Done with messi
----------------------------------
Working on lebron
Getting IG data
Getting TW data
Done with lebron
----------------------------------
Working on nadal
Getting IG data
Getting TW data
Done with nadal
----------------------------------
Working on mayweather
Getting IG data
Getting TW data
Done with mayweather
----------------------------------
Working on FalcaoGarcia
Getting IG data
Done with FalcaoGarcia
----------------------------------


In [51]:
users_data

[{'id': 1,
  'name': 'cristiano',
  'instagram_username': 'cristiano',
  'twitter_username': 'Cristiano',
  'instagram_data': {'username': 'cristiano',
   'user_id_instagram': '173560420',
   'n_followers': 505680928,
   'n_following': 523,
   'n_posts_total': 3406,
   'n_posts_retrieved': 10,
   'n_likes_total': 119441995,
   'n_likes_retrieved': 119441995,
   'n_comments_total': 1043854,
   'n_comments_retrieved': 151,
   'created_at': '2022-11-29T09:53:00.713671',
   'sentiment_instagram': 0.6281752879221326},
  'twitter_data': {'username': 'Cristiano',
   'n_followers': 105459429,
   'n_retweets': 0,
   'n_tweets': 5,
   'n_retweets_to_user': 177527,
   'n_favorites_to_user': 2254962,
   'n_replies_to_user': 12,
   'created_at': '2022-11-29T09:53:20.284805',
   'sentiment_twitter': 0.34042454919233583}},
 {'id': 2,
  'name': 'messi',
  'instagram_username': 'leomessi',
  'twitter_username': 'Ieomessiok',
  'instagram_data': {'username': 'leomessi',
   'user_id_instagram': '42755389

### Compute percentiles and SPIs

In [66]:
def compute_spi_ig(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_posts_total": 0.1, 
        "n_likes_retrieved": 0.2, 
        "n_comments_retrieved": 0.1, 
        "sentiment_instagram": 0.3, 
        "relative_engagement": 0.2
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_ig_data(data):
    users_data_instagram = []
    for user_data in data:
        if ("instagram_data" in user_data.keys()):
            dict_data = user_data["instagram_data"].copy()
            dict_data["user_id"] = user_data["id"]
            users_data_instagram.append(dict_data)
        else:
            empty_dict = {'user_id': users_data["id"],
                        'username': '',
                        "user_id_instagram": '',
                        'n_followers': 0,
                        'n_following': 0,
                        'n_posts_total': 0,
                        'n_posts_retrieved': 0,
                        'n_likes_total': 0,
                        'n_likes_retrieved': 0,
                        'n_comments_total': 0,
                        'n_comments_retrieved': 0,
                        'created_at': '',
                        'sentiment_instagram': 0}
            users_data_instagram.append(empty_dict)
        
    df_ig = pd.DataFrame(users_data_instagram)
    df_ig["relative_engagement"] = np.clip((df_ig["n_likes_retrieved"] + df_ig["n_comments_retrieved"])/(df_ig["n_followers"])*100, 0, 100)
    selected_cols = ["n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    df_ig_perc = df_ig.copy()
    for col in selected_cols:
        df_ig_perc[col] = df_ig_perc[col].apply(lambda x: stats.percentileofscore(df_ig_perc[col], x))
    df_ig_perc["spi"] = df_ig_perc[selected_cols].apply(compute_spi_ig, axis=1)
    spi_ig_columns = ["user_id", "username", "n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    return pd.concat([df_ig[spi_ig_columns], df_ig_perc["spi"], df_ig["created_at"]], axis=1)



In [71]:
processed_ig_data = process_ig_data(users_data)
processed_ig_data

,user_id,username,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,1,cristiano,505680928,3406,119441995,151,0.628175,23.620061,78.333333,2022-11-29T09:53:00.713671
1,2,leomessi,381256979,951,102789428,369,0.647725,26.960765,80.000000,2022-11-29T09:53:58.009076
2,3,lebron,663958,2961,119812,332,0.623161,18.095120,45.000000,2022-11-29T09:55:09.329696
3,4,rafaelnadal,17109255,1249,2091180,355,0.719216,12.224583,70.000000,2022-11-29T09:56:23.588267
4,5,floydmayweather,28511573,1043,1867545,383,0.586477,6.551473,41.666667,2022-11-29T09:57:37.930404
5,6,falcao,15978091,1325,1012447,333,0.597372,6.338554,35.000000,2022-11-29T09:58:50.943299


In [67]:
def compute_spi_tw(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_retweets": 0.1, 
        "n_tweets": 0.1, 
        "n_retweets_to_user": 0.1, 
        "n_favorites_to_user": 0.1, 
        "n_replies_to_user": 0.1,
        "sentiment_twitter": 0.3,
        "relative_engagement": 0.1
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_tw_data(data):
    users_data_twitter = []
    for user_data in data:
        if ("twitter_data" in user_data.keys()):
            dict_data = user_data["twitter_data"].copy()
            dict_data["user_id"] = user_data["id"]
            users_data_twitter.append(dict_data)
        else:
            empty_dict = {'user_id': user_data["id"],
                        'username': '',
                        "n_followers": 0,
                        'n_retweets': 0,
                        'n_tweets': 0,
                        'n_retweets_to_user': 0,
                        'n_favorites_to_user': 0,
                        'n_replies_to_user': 0,
                        'created_at': dt.now(),
                        'sentiment_twitter': 0}
            users_data_twitter.append(empty_dict)
    
    df_tw = pd.DataFrame(users_data_twitter)
    df_tw["relative_engagement"] = (df_tw.apply(lambda x: (x["n_favorites_to_user"] + x["n_retweets_to_user"] + x["n_replies_to_user"])/(x["n_followers"])*100 if x["n_tweets"] > 0 else 0, axis=1))
    selected_cols = ["n_followers", "n_retweets", "n_tweets", "n_retweets_to_user", "n_favorites_to_user", "n_replies_to_user", "sentiment_twitter", "relative_engagement"]
    df_tw_perc = df_tw.copy()
    for col in selected_cols:
        df_tw_perc[col] = df_tw_perc[col].apply(lambda x: stats.percentileofscore(df_tw_perc[col], x))
    df_tw_perc["spi"] = df_tw_perc[selected_cols].apply(compute_spi_tw, axis=1)
    spi_tw_columns = ["user_id","username","n_followers","n_retweets","n_tweets","n_retweets_to_user","n_favorites_to_user","n_replies_to_user","sentiment_twitter","relative_engagement"]
    return pd.concat([df_tw[spi_tw_columns], df_tw_perc["spi"], df_tw["created_at"]], axis=1)

In [70]:
processed_tw_data = process_tw_data(users_data)
processed_tw_data

,user_id,username,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,1,Cristiano,105459429,0,5,177527,2254962,12,0.340425,2.306575,75.000000,2022-11-29T09:53:20.284805
1,2,Ieomessiok,594994,0,3,25125,337260,13,0.419460,60.907841,76.666667,2022-11-29T09:54:34.340808
2,3,KingJames,52484753,0,5,5210,91812,13,0.353084,0.184882,72.500000,2022-11-29T09:55:44.714440
3,4,RafaelNadal,15822202,0,1,183,1681,2,0.597482,0.011794,67.500000,2022-11-29T09:57:00.129607
4,5,FloydMayweather,7831564,0,0,0,0,0,0.200000,0.000000,33.333333,2022-11-29T09:58:15.400783
5,6,,0,0,0,0,0,0,0.000000,0.000000,25.000000,2022-11-29 10:56:41.069773


### Check if there are any records in instagram table or twitter table for each user

If there are no records the spi stays the same. Else, spi bonus is computed and added to spi

#### Instagram

In [252]:
def input_ig_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO instagram_stats(user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["username"], record_dict["n_followers"], record_dict["n_posts_total"], record_dict["n_likes_retrieved"], record_dict["n_comments_retrieved"], record_dict["sentiment_instagram"], record_dict["relative_engagement"], record_dict["spi"]))

def retrieve_latest_record_ig(user_id):
    cur.execute(f"SELECT * FROM instagram_stats WHERE user_id = {user_id} ORDER BY created_at DESC LIMIT 1;") 
    for record_id, user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at in cur:
        record_dict = {
            "user_id": user_id,
            "username": instagram_username,
            "n_followers": n_followers,
            "n_posts_total": n_posts_total,
            "n_likes_retrieved": n_likes_retrieved,
            "n_comments_retrieved": n_comments_retrieved,
            "sentiment_instagram": sentiment,
            "relative_engagement": engagement,
            "spi": spi,
            "created_at": created_at
        }
    return pd.DataFrame(record_dict, index=[0])
      

def compute_definitive_ig_spi(user_id, new_record, bonus_weight=0.15):
    cur.execute(f"SELECT COUNT(*) FROM instagram_stats WHERE user_id = {user_id};") 
    for result in cur: 
        count_records = result[0]
    if count_records == 0:
        return new_record
    else: 
        
        dict_change = {}
        num_cols = [col for col in new_record.columns if is_numeric_dtype(new_record[col])]
        dict_change["username"] = new_record["username"].unique()[0]

        latest_record_db = retrieve_latest_record_ig(user_id)
        
        for col in num_cols:
            dict_change[col] = new_record.loc[0][col] - latest_record_db.loc[0][col]
            
        df_change = pd.DataFrame(dict_change, index=[0])
        
        dict_relative_change = {}
        dict_change = df_change.to_dict('records')[0]
        dict_base = latest_record_db.loc[0].to_dict()

        for key in dict_change.keys():
            if key == "username":
                dict_relative_change[key] = dict_change[key]
            elif key == "created_at" or key == "spi":
                pass
            else:
                if dict_base[key] == 0 and dict_change[key] != 0:
                    dict_relative_change[key] = 1
                elif dict_base[key] == 0 and dict_change[key] == 0:
                    dict_relative_change[key] = 0
                else:
                    dict_relative_change[key] = dict_change[key] / dict_base[key]
                    
        dict_ig_spi_bonus_weight = {
            'n_followers': 0.1,
            'n_posts_total': 0.1,
            'n_likes_retrieved': 0.1,
            'n_comments_retrieved': 0.1,
            'sentiment_instagram': 0.4,
            'relative_engagement': 0.2
        }

        bonus_spi = 0
        for k in dict_relative_change.keys():
            if k != "username" and k != "user_id":
                bonus_spi += dict_ig_spi_bonus_weight[k]*dict_relative_change[k]

        new_record_upd = new_record.copy()
        new_record_upd.loc[0, "spi"] = new_record_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100

        return new_record_upd[:1]


In [254]:
processed_ig_data

,user_id,username,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,1,cristiano,505680928,3406,119441995,151,0.628175,23.620061,78.333333,2022-11-29T09:53:00.713671
1,2,leomessi,381256979,951,102789428,369,0.647725,26.960765,80.000000,2022-11-29T09:53:58.009076
2,3,lebron,663958,2961,119812,332,0.623161,18.095120,45.000000,2022-11-29T09:55:09.329696
3,4,rafaelnadal,17109255,1249,2091180,355,0.719216,12.224583,70.000000,2022-11-29T09:56:23.588267
4,5,floydmayweather,28511573,1043,1867545,383,0.586477,6.551473,41.666667,2022-11-29T09:57:37.930404
5,6,falcao,15978091,1325,1012447,333,0.597372,6.338554,35.000000,2022-11-29T09:58:50.943299


In [257]:
df_updated_ig = pd.DataFrame()
for user_id in processed_ig_data["user_id"].unique():
    new_record = processed_ig_data[processed_ig_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    df_updated_ig = pd.concat([df_updated_ig, compute_definitive_ig_spi(user_id, new_record)], axis=0, ignore_index=True)
df_updated_ig

,user_id,username,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,1,cristiano,505680928,3406,119441995,151,0.628175,23.620061,78.333331,2022-11-29T09:53:00.713671
1,2,leomessi,381256979,951,102789428,369,0.647725,26.960765,79.999993,2022-11-29T09:53:58.009076
2,3,lebron,663958,2961,119812,332,0.623161,18.095120,45.000004,2022-11-29T09:55:09.329696
3,4,rafaelnadal,17109255,1249,2091180,355,0.719216,12.224583,69.999998,2022-11-29T09:56:23.588267
4,5,floydmayweather,28511573,1043,1867545,383,0.586477,6.551473,41.666670,2022-11-29T09:57:37.930404
5,6,falcao,15978091,1325,1012447,333,0.597372,6.338554,35.000006,2022-11-29T09:58:50.943299


This is the flow evey night at 12am:

1. Query the usernames of social media in users table
2. Extract data from social media apis
3. Compute percentiles and spis
4. If there are existing records of the users in the social media tables, spi bonus is computed 
and added to base spi. Else, no bonus is added, spi stays the same
5. spis of all social media are computed into a general SPI per user
6. Input the data to the database (instagram_stats, twitter_stats, spis)

In [258]:
df_ig_fully_proc = pd.DataFrame()
for user_id in processed_ig_data["user_id"].unique():
    new_record = processed_ig_data[processed_ig_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    record_updated = compute_definitive_ig_spi(user_id, new_record)
    df_ig_fully_proc = pd.concat([df_ig_fully_proc, record_updated], axis=0, ignore_index=True)
    input_ig_data(cur, record_updated)

df_ig_fully_proc

,user_id,username,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,1,cristiano,505680928,3406,119441995,151,0.628175,23.620061,78.333331,2022-11-29T09:53:00.713671
1,2,leomessi,381256979,951,102789428,369,0.647725,26.960765,79.999993,2022-11-29T09:53:58.009076
2,3,lebron,663958,2961,119812,332,0.623161,18.095120,45.000004,2022-11-29T09:55:09.329696
3,4,rafaelnadal,17109255,1249,2091180,355,0.719216,12.224583,69.999998,2022-11-29T09:56:23.588267
4,5,floydmayweather,28511573,1043,1867545,383,0.586477,6.551473,41.666670,2022-11-29T09:57:37.930404
5,6,falcao,15978091,1325,1012447,333,0.597372,6.338554,35.000006,2022-11-29T09:58:50.943299


In [260]:
conn.commit()

In [259]:
cur.execute("SELECT * FROM instagram_stats;")

for record_id , user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at in cur: 
    print(record_id , user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at)

25 1 cristiano 505680928 3406 119441995 151 0.628175 23.6201 78.3333 2022-11-29 12:31:22
26 2 leomessi 381256979 951 102789428 369 0.647725 26.9608 80.0 2022-11-29 12:31:22
27 3 lebron 663958 2961 119812 332 0.623161 18.0951 45.0 2022-11-29 12:31:22
28 4 rafaelnadal 17109255 1249 2091180 355 0.719216 12.2246 70.0 2022-11-29 12:31:22
29 5 floydmayweather 28511573 1043 1867545 383 0.586477 6.55147 41.6667 2022-11-29 12:31:22
30 6 falcao 15978091 1325 1012447 333 0.597372 6.33855 35.0 2022-11-29 12:31:22
31 1 cristiano 505680928 3406 119441995 151 0.628175 23.6201 78.3333 2022-11-29 13:52:16
32 1 cristiano 505680928 3406 119441995 151 0.628175 23.6201 78.3333 2022-11-29 13:53:28
33 2 leomessi 381256979 951 102789428 369 0.647725 26.9608 80.0 2022-11-29 13:53:28
34 3 lebron 663958 2961 119812 332 0.623161 18.0951 45.0 2022-11-29 13:53:28
35 4 rafaelnadal 17109255 1249 2091180 355 0.719216 12.2246 70.0 2022-11-29 13:53:28
36 5 floydmayweather 28511573 1043 1867545 383 0.586477 6.55147 41.66

In [162]:
conn.close()

try:
    conn = mariadb.connect(
        user="root",
        password="itsense",
        host="localhost",
        port=3306,
        database="vertikal_ds"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

#### Twitter

In [286]:
def input_tw_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO twitter_stats(user_id, twitter_username, n_followers, n_retweets , n_tweets , n_retweets_to_user , n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["username"], record_dict["n_followers"], record_dict["n_retweets"], record_dict["n_tweets"], record_dict["n_retweets_to_user"], record_dict["n_favorites_to_user"], record_dict["n_replies_to_user"], record_dict["sentiment_twitter"], record_dict["relative_engagement"], record_dict["spi"]))

def retrieve_latest_record_tw(user_id):
    cur.execute(f"SELECT * FROM twitter_stats WHERE user_id = {user_id} ORDER BY created_at DESC LIMIT 1;") 
    for record_id, user_id, twitter_username, n_followers, n_retweets, n_tweets, n_retweets_to_user, n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at in cur:
        record_dict = {
            "user_id": user_id,
            "username": twitter_username,
            "n_followers": n_followers,
            "n_retweets": n_retweets,
            "n_tweets": n_tweets,
            "n_retweets_to_user": n_retweets_to_user,
            "n_favorites_to_user": n_favorites_to_user,
            "n_replies_to_user": n_replies_to_user,
            "sentiment_twitter": sentiment,
            "relative_engagement": engagement,
            "spi": spi,
            "created_at": created_at
        }
    return pd.DataFrame(record_dict, index=[0])
      

def compute_definitive_tw_spi(user_id, new_record, bonus_weight=0.15):
    cur.execute(f"SELECT COUNT(*) FROM twitter_stats WHERE user_id = {user_id};") 
    for result in cur: 
        count_records = result[0]
    if count_records == 0:
        return new_record
    else: 
        
        dict_change = {}
        num_cols = [col for col in new_record.columns if is_numeric_dtype(new_record[col])]
        dict_change["username"] = new_record["username"].unique()[0]

        latest_record_db = retrieve_latest_record_tw(user_id)
        for col in num_cols:
            dict_change[col] = new_record.loc[0][col] - latest_record_db.loc[0][col]
            
        df_change = pd.DataFrame(dict_change, index=[0])
        
        dict_relative_change = {}
        dict_change = df_change.to_dict('records')[0]
        dict_base = latest_record_db.loc[0].to_dict()

        for key in dict_change.keys():
            if key == "username":
                dict_relative_change[key] = dict_change[key]
            elif key == "created_at" or key == "spi":
                pass
            else:
                if dict_base[key] == 0 and dict_change[key] != 0:
                    dict_relative_change[key] = 1
                elif dict_base[key] == 0 and dict_change[key] == 0:
                    dict_relative_change[key] = 0
                else:
                    dict_relative_change[key] = dict_change[key] / dict_base[key]
                    
        dict_tw_spi_bonus_weight = {
        'n_followers': 0.1,
        'n_retweets': 0.1,
        'n_tweets': 0.1,
        'n_retweets_to_user': 0.1,
        'n_favorites_to_user': 0.1,
        'n_replies_to_user': 0.1,
        'sentiment_twitter': 0.3,
        'relative_engagement': 0.1
    }

        bonus_spi = 0
        for k in dict_relative_change.keys():
            if k != "username" and k != "user_id":
                bonus_spi += dict_tw_spi_bonus_weight[k]*dict_relative_change[k]

        new_record_upd = new_record.copy()
        new_record_upd.loc[0, "spi"] = new_record_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100

        return new_record_upd[:1]


In [287]:
processed_tw_data

,user_id,username,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,1,Cristiano,105459429,0,5,177527,2254962,12,0.340425,2.306575,75.000000,2022-11-29T09:53:20.284805
1,2,Ieomessiok,594994,0,3,25125,337260,13,0.419460,60.907841,76.666667,2022-11-29T09:54:34.340808
2,3,KingJames,52484753,0,5,5210,91812,13,0.353084,0.184882,72.500000,2022-11-29T09:55:44.714440
3,4,RafaelNadal,15822202,0,1,183,1681,2,0.597482,0.011794,67.500000,2022-11-29T09:57:00.129607
4,5,FloydMayweather,7831564,0,0,0,0,0,0.200000,0.000000,33.333333,2022-11-29T09:58:15.400783
5,6,,0,0,0,0,0,0,0.000000,0.000000,25.000000,2022-11-29 10:56:41.069773


In [289]:
df_tw_fully_proc = pd.DataFrame()
for user_id in processed_tw_data["user_id"].unique():
    new_record = processed_tw_data[processed_tw_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    record_updated = compute_definitive_tw_spi(user_id, new_record)
    df_tw_fully_proc = pd.concat([df_tw_fully_proc, record_updated], axis=0, ignore_index=True)
    input_tw_data(cur, record_updated)

df_tw_fully_proc

,user_id,username,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,1,Cristiano,105459429,0,5,177527,2254962,12,0.340425,2.306575,74.999991,2022-11-29T09:53:20.284805
1,2,Ieomessiok,594994,0,3,25125,337260,13,0.419460,60.907841,76.666671,2022-11-29T09:54:34.340808
2,3,KingJames,52484753,0,5,5210,91812,13,0.353084,0.184882,72.500002,2022-11-29T09:55:44.714440
3,4,RafaelNadal,15822202,0,1,183,1681,2,0.597482,0.011794,67.499998,2022-11-29T09:57:00.129607
4,5,FloydMayweather,7831564,0,0,0,0,0,0.200000,0.000000,33.333333,2022-11-29T09:58:15.400783
5,6,,0,0,0,0,0,0,0.000000,0.000000,25.000000,2022-11-29 10:56:41.069773


In [ ]:
cur.execute("SELECT * FROM twitter_stats ORDER BY record_id;")

for record_id, user_id, twitter_username, n_followers, n_retweets, n_tweets, n_retweets_to_user, n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at in cur: 
    print(record_id, user_id, twitter_username, n_followers, n_retweets, n_tweets, n_retweets_to_user, n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at)

In [ ]:
conn.commit()

-----

## Putting all together

Connect to database

In [309]:
try:
    conn = mariadb.connect(
        user="root",
        password="itsense",
        host="localhost",
        port=3306,
        database="vertikal_ds"
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

Retrieve users and social media usernames

In [ ]:
cur.execute("SELECT * FROM users;")

ids = []
names = []
instagram_usernames = []
twitter_usernames = []

for id, name, instagram_username, twitter_username, last_update in cur: 
    ids.append(id)
    names.append(name)
    instagram_usernames.append(instagram_username)
    twitter_usernames.append(twitter_username)
    print(f"name: {name}, instagram_username: {instagram_username}, twitter_username: {twitter_username}, last_update: {last_update}")

Extract the data for all the users

In [ ]:
url_ig = "http://localhost:5000/api/instagram"
url_tw = "http://localhost:5000/api/twitter"

users_data = []

for ix, id in enumerate(ids):
    user_data_dict = {}
    user_data_dict["id"] = id
    user_data_dict["name"] = names[ix]
    user_data_dict["instagram_username"] = instagram_usernames[ix]
    user_data_dict["twitter_username"] = twitter_usernames[ix]
    print(f"Working on {names[ix]}")

    if user_data_dict["instagram_username"] != None:
        
        print("Getting IG data")
        body = {"username": user_data_dict["instagram_username"]}
        r = requests.post(url_ig, json=body)
        response = r.json()
        user_data_dict["instagram_data"] = response

    if user_data_dict["twitter_username"] != None:
        print("Getting TW data")
        body = {"username": user_data_dict["twitter_username"]}
        r = requests.post(url_tw, json=body)
        response = r.json()
        user_data_dict["twitter_data"] = response
    
    users_data.append(user_data_dict)
    print(f"Done with {names[ix]}")
    print("----------------------------------")

Process the data, compute engagement, percentiles and spi

In [ ]:
def compute_spi_ig(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_posts_total": 0.1, 
        "n_likes_retrieved": 0.2, 
        "n_comments_retrieved": 0.1, 
        "sentiment_instagram": 0.3, 
        "relative_engagement": 0.2
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_ig_data(data):
    users_data_instagram = []
    for user_data in data:
        if ("instagram_data" in user_data.keys()):
            dict_data = user_data["instagram_data"].copy()
            dict_data["user_id"] = user_data["id"]
            users_data_instagram.append(dict_data)
        else:
            empty_dict = {'user_id': users_data["id"],
                        'username': '',
                        "user_id_instagram": '',
                        'n_followers': 0,
                        'n_following': 0,
                        'n_posts_total': 0,
                        'n_posts_retrieved': 0,
                        'n_likes_total': 0,
                        'n_likes_retrieved': 0,
                        'n_comments_total': 0,
                        'n_comments_retrieved': 0,
                        'created_at': '',
                        'sentiment_instagram': 0}
            users_data_instagram.append(empty_dict)
        
    df_ig = pd.DataFrame(users_data_instagram)
    df_ig["relative_engagement"] = np.clip((df_ig["n_likes_retrieved"] + df_ig["n_comments_retrieved"])/(df_ig["n_followers"])*100, 0, 100)
    selected_cols = ["n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    df_ig_perc = df_ig.copy()
    for col in selected_cols:
        df_ig_perc[col] = df_ig_perc[col].apply(lambda x: stats.percentileofscore(df_ig_perc[col], x))
    df_ig_perc["spi"] = df_ig_perc[selected_cols].apply(compute_spi_ig, axis=1)
    spi_ig_columns = ["user_id", "username", "n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    return pd.concat([df_ig[spi_ig_columns], df_ig_perc["spi"], df_ig["created_at"]], axis=1)

def compute_spi_tw(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_retweets": 0.1, 
        "n_tweets": 0.1, 
        "n_retweets_to_user": 0.1, 
        "n_favorites_to_user": 0.1, 
        "n_replies_to_user": 0.1,
        "sentiment_twitter": 0.3,
        "relative_engagement": 0.1
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_tw_data(data):
    users_data_twitter = []
    for user_data in data:
        if ("twitter_data" in user_data.keys()):
            dict_data = user_data["twitter_data"].copy()
            dict_data["user_id"] = user_data["id"]
            users_data_twitter.append(dict_data)
        else:
            empty_dict = {'user_id': user_data["id"],
                        'username': '',
                        "n_followers": 0,
                        'n_retweets': 0,
                        'n_tweets': 0,
                        'n_retweets_to_user': 0,
                        'n_favorites_to_user': 0,
                        'n_replies_to_user': 0,
                        'created_at': dt.now(),
                        'sentiment_twitter': 0}
            users_data_twitter.append(empty_dict)
    
    df_tw = pd.DataFrame(users_data_twitter)
    df_tw["relative_engagement"] = (df_tw.apply(lambda x: (x["n_favorites_to_user"] + x["n_retweets_to_user"] + x["n_replies_to_user"])/(x["n_followers"])*100 if x["n_tweets"] > 0 else 0, axis=1))
    selected_cols = ["n_followers", "n_retweets", "n_tweets", "n_retweets_to_user", "n_favorites_to_user", "n_replies_to_user", "sentiment_twitter", "relative_engagement"]
    df_tw_perc = df_tw.copy()
    for col in selected_cols:
        df_tw_perc[col] = df_tw_perc[col].apply(lambda x: stats.percentileofscore(df_tw_perc[col], x))
    df_tw_perc["spi"] = df_tw_perc[selected_cols].apply(compute_spi_tw, axis=1)
    spi_tw_columns = ["user_id","username","n_followers","n_retweets","n_tweets","n_retweets_to_user","n_favorites_to_user","n_replies_to_user","sentiment_twitter","relative_engagement"]
    return pd.concat([df_tw[spi_tw_columns], df_tw_perc["spi"], df_tw["created_at"]], axis=1)


In [ ]:
processed_ig_data = process_ig_data(users_data)
processed_tw_data = process_tw_data(users_data)

Compute spi bonus

In [ ]:
def input_ig_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO instagram_stats(user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["username"], record_dict["n_followers"], record_dict["n_posts_total"], record_dict["n_likes_retrieved"], record_dict["n_comments_retrieved"], record_dict["sentiment_instagram"], record_dict["relative_engagement"], record_dict["spi"]))<

def retrieve_latest_record_ig(user_id):
    cur.execute(f"SELECT * FROM instagram_stats WHERE user_id = {user_id} ORDER BY created_at DESC LIMIT 1;") 
    for record_id, user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at in cur:
        record_dict = {
            "user_id": user_id,
            "username": instagram_username,
            "n_followers": n_followers,
            "n_posts_total": n_posts_total,
            "n_likes_retrieved": n_likes_retrieved,
            "n_comments_retrieved": n_comments_retrieved,
            "sentiment_instagram": sentiment,
            "relative_engagement": engagement,
            "spi": spi,
            "created_at": created_at
        }
    return pd.DataFrame(record_dict, index=[0])
      

def compute_definitive_ig_spi(user_id, new_record, bonus_weight=0.15):
    cur.execute(f"SELECT COUNT(*) FROM instagram_stats WHERE user_id = {user_id};") 
    for result in cur: 
        count_records = result[0]
    if count_records == 0:
        return new_record
    else: 
        
        dict_change = {}
        num_cols = [col for col in new_record.columns if is_numeric_dtype(new_record[col])]
        dict_change["username"] = new_record["username"].unique()[0]

        latest_record_db = retrieve_latest_record_ig(user_id)
        
        for col in num_cols:
            dict_change[col] = new_record.loc[0][col] - latest_record_db.loc[0][col]
            
        df_change = pd.DataFrame(dict_change, index=[0])
        
        dict_relative_change = {}
        dict_change = df_change.to_dict('records')[0]
        dict_base = latest_record_db.loc[0].to_dict()

        for key in dict_change.keys():
            if key == "username":
                dict_relative_change[key] = dict_change[key]
            elif key == "created_at" or key == "spi":
                pass
            else:
                if dict_base[key] == 0 and dict_change[key] != 0:
                    dict_relative_change[key] = 1
                elif dict_base[key] == 0 and dict_change[key] == 0:
                    dict_relative_change[key] = 0
                else:
                    dict_relative_change[key] = dict_change[key] / dict_base[key]
                    
        dict_ig_spi_bonus_weight = {
            'n_followers': 0.1,
            'n_posts_total': 0.1,
            'n_likes_retrieved': 0.1,
            'n_comments_retrieved': 0.1,
            'sentiment_instagram': 0.4,
            'relative_engagement': 0.2
        }

        bonus_spi = 0
        for k in dict_relative_change.keys():
            if k != "username" and k != "user_id":
                bonus_spi += dict_ig_spi_bonus_weight[k]*dict_relative_change[k]

        new_record_upd = new_record.copy()
        new_record_upd.loc[0, "spi"] = new_record_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100

        return new_record_upd[:1]

def input_tw_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO twitter_stats(user_id, twitter_username, n_followers, n_retweets , n_tweets , n_retweets_to_user , n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["username"], record_dict["n_followers"], record_dict["n_retweets"], record_dict["n_tweets"], record_dict["n_retweets_to_user"], record_dict["n_favorites_to_user"], record_dict["n_replies_to_user"], record_dict["sentiment_twitter"], record_dict["relative_engagement"], record_dict["spi"]))

def retrieve_latest_record_tw(user_id):
    cur.execute(f"SELECT * FROM twitter_stats WHERE user_id = {user_id} ORDER BY created_at DESC LIMIT 1;") 
    for record_id, user_id, twitter_username, n_followers, n_retweets, n_tweets, n_retweets_to_user, n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at in cur:
        record_dict = {
            "user_id": user_id,
            "username": twitter_username,
            "n_followers": n_followers,
            "n_retweets": n_retweets,
            "n_tweets": n_tweets,
            "n_retweets_to_user": n_retweets_to_user,
            "n_favorites_to_user": n_favorites_to_user,
            "n_replies_to_user": n_replies_to_user,
            "sentiment_twitter": sentiment,
            "relative_engagement": engagement,
            "spi": spi,
            "created_at": created_at
        }
    return pd.DataFrame(record_dict, index=[0])
      

def compute_definitive_tw_spi(user_id, new_record, bonus_weight=0.15):
    cur.execute(f"SELECT COUNT(*) FROM twitter_stats WHERE user_id = {user_id};") 
    for result in cur: 
        count_records = result[0]
    if count_records == 0:
        return new_record
    else: 
        
        dict_change = {}
        num_cols = [col for col in new_record.columns if is_numeric_dtype(new_record[col])]
        dict_change["username"] = new_record["username"].unique()[0]

        latest_record_db = retrieve_latest_record_tw(user_id)
        for col in num_cols:
            dict_change[col] = new_record.loc[0][col] - latest_record_db.loc[0][col]
            
        df_change = pd.DataFrame(dict_change, index=[0])
        
        dict_relative_change = {}
        dict_change = df_change.to_dict('records')[0]
        dict_base = latest_record_db.loc[0].to_dict()

        for key in dict_change.keys():
            if key == "username":
                dict_relative_change[key] = dict_change[key]
            elif key == "created_at" or key == "spi":
                pass
            else:
                if dict_base[key] == 0 and dict_change[key] != 0:
                    dict_relative_change[key] = 1
                elif dict_base[key] == 0 and dict_change[key] == 0:
                    dict_relative_change[key] = 0
                else:
                    dict_relative_change[key] = dict_change[key] / dict_base[key]
                    
        dict_tw_spi_bonus_weight = {
        'n_followers': 0.1,
        'n_retweets': 0.1,
        'n_tweets': 0.1,
        'n_retweets_to_user': 0.1,
        'n_favorites_to_user': 0.1,
        'n_replies_to_user': 0.1,
        'sentiment_twitter': 0.3,
        'relative_engagement': 0.1
    }

        bonus_spi = 0
        for k in dict_relative_change.keys():
            if k != "username" and k != "user_id":
                bonus_spi += dict_tw_spi_bonus_weight[k]*dict_relative_change[k]

        new_record_upd = new_record.copy()
        new_record_upd.loc[0, "spi"] = new_record_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100

        return new_record_upd[:1]


Compute general SPI and input the data into the database

In [296]:
df_ig_fully_proc = pd.DataFrame()
for user_id in processed_ig_data["user_id"].unique():
    new_record = processed_ig_data[processed_ig_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    record_updated = compute_definitive_ig_spi(user_id, new_record)
    df_ig_fully_proc = pd.concat([df_ig_fully_proc, record_updated], axis=0, ignore_index=True)
    # input_ig_data(cur, record_updated)

df_ig_fully_proc

,user_id,username,n_followers,n_posts_total,n_likes_retrieved,n_comments_retrieved,sentiment_instagram,relative_engagement,spi,created_at
0,1,cristiano,505680928,3406,119441995,151,0.628175,23.620061,78.333331,2022-11-29T09:53:00.713671
1,2,leomessi,381256979,951,102789428,369,0.647725,26.960765,79.999993,2022-11-29T09:53:58.009076
2,3,lebron,663958,2961,119812,332,0.623161,18.095120,45.000004,2022-11-29T09:55:09.329696
3,4,rafaelnadal,17109255,1249,2091180,355,0.719216,12.224583,69.999998,2022-11-29T09:56:23.588267
4,5,floydmayweather,28511573,1043,1867545,383,0.586477,6.551473,41.666670,2022-11-29T09:57:37.930404
5,6,falcao,15978091,1325,1012447,333,0.597372,6.338554,35.000006,2022-11-29T09:58:50.943299


In [297]:
df_tw_fully_proc = pd.DataFrame()
for user_id in processed_tw_data["user_id"].unique():
    new_record = processed_tw_data[processed_tw_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    record_updated = compute_definitive_tw_spi(user_id, new_record)
    df_tw_fully_proc = pd.concat([df_tw_fully_proc, record_updated], axis=0, ignore_index=True)
    # input_tw_data(cur, record_updated)

df_tw_fully_proc

,user_id,username,n_followers,n_retweets,n_tweets,n_retweets_to_user,n_favorites_to_user,n_replies_to_user,sentiment_twitter,relative_engagement,spi,created_at
0,1,Cristiano,105459429,0,5,177527,2254962,12,0.340425,2.306575,74.999991,2022-11-29T09:53:20.284805
1,2,Ieomessiok,594994,0,3,25125,337260,13,0.419460,60.907841,76.666671,2022-11-29T09:54:34.340808
2,3,KingJames,52484753,0,5,5210,91812,13,0.353084,0.184882,72.500002,2022-11-29T09:55:44.714440
3,4,RafaelNadal,15822202,0,1,183,1681,2,0.597482,0.011794,67.499998,2022-11-29T09:57:00.129607
4,5,FloydMayweather,7831564,0,0,0,0,0,0.200000,0.000000,33.333333,2022-11-29T09:58:15.400783
5,6,,0,0,0,0,0,0,0.000000,0.000000,25.000000,2022-11-29 10:56:41.069773


In [314]:
def input_spi_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO spis(user_id, spi, spi_instagram, spi_twitter, created_at) VALUES (?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["spi"], record_dict["spi_instagram"], record_dict["spi_twitter"]))

In [305]:
spi_df = pd.merge(df_ig_fully_proc[["user_id", "spi"]], df_tw_fully_proc[["user_id", "spi"]], on="user_id", suffixes=["_instagram", "_twitter"])
weight_ig = 0.7
weight_tw = 0.3
spi_df["spi"] = spi_df["spi_instagram"] * weight_ig +  spi_df["spi_twitter"] * weight_tw
spi_df

,user_id,spi_instagram,spi_twitter,spi
0,1,78.333331,74.999991,77.333329
1,2,79.999993,76.666671,78.999996
2,3,45.000004,72.500002,53.250003
3,4,69.999998,67.499998,69.249998
4,5,41.666670,33.333333,39.166669
5,6,35.000006,25.000000,32.000004


In [315]:
for user_id in spi_df["user_id"].unique():
    new_record = spi_df[spi_df["user_id"] == user_id].reset_index().drop("index", axis=1)
    input_spi_data(cur, new_record)

{'user_id': 1, 'spi_instagram': 78.33333118880616, 'spi_twitter': 74.99999089755802, 'spi': 77.33332910143172}
{'user_id': 2, 'spi_instagram': 79.99999259689243, 'spi_twitter': 76.66667138736939, 'spi': 78.9999962340355}
{'user_id': 3, 'spi_instagram': 45.000003572894244, 'spi_twitter': 72.50000161741232, 'spi': 53.25000298624967}
{'user_id': 4, 'spi_instagram': 69.9999977048587, 'spi_twitter': 67.49999810891453, 'spi': 69.24999782607544}
{'user_id': 5, 'spi_instagram': 41.66667043228286, 'spi_twitter': 33.333333333333336, 'spi': 39.166669302597995}
{'user_id': 6, 'spi_instagram': 35.00000570077124, 'spi_twitter': 25.0, 'spi': 32.00000399053987}


In [316]:
cur.execute("SELECT * FROM spis;")

for record_id , user_id, spi, spi_instagram, spi_twitter, created_at in cur: 
    print(record_id , user_id, spi, spi_instagram, spi_twitter, created_at)

1 1 77.3333 78.3333 75.0 2022-11-29 14:44:17
2 2 79.0 80.0 76.6667 2022-11-29 14:44:17
3 3 53.25 45.0 72.5 2022-11-29 14:44:17
4 4 69.25 70.0 67.5 2022-11-29 14:44:17
5 5 39.1667 41.6667 33.3333 2022-11-29 14:44:17
6 6 32.0 35.0 25.0 2022-11-29 14:44:17


In [317]:
conn.commit()

In [318]:
conn.close()

---

In [322]:
import requests
import mariadb
import sys
import pandas       as pd
import numpy        as np
from datetime           import datetime     as dt
from scipy              import stats
from pandas.api.types   import is_numeric_dtype

# --- Connect to database ---
try:
    conn = mariadb.connect(
        user="root",
        password="itsense",
        host="localhost",
        port=3306,
        database="vertikal_ds"

    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

# Get Cursor
cur = conn.cursor()

# --- Retrieve users and social media usernames ---
cur.execute("SELECT * FROM users;")

ids = []
names = []
instagram_usernames = []
twitter_usernames = []

for id, name, instagram_username, twitter_username, last_update in cur: 
    ids.append(id)
    names.append(name)
    instagram_usernames.append(instagram_username)
    twitter_usernames.append(twitter_username)
    print(f"name: {name}, instagram_username: {instagram_username}, twitter_username: {twitter_username}, last_update: {last_update}")

# --- Extract the data for all the users ---

url_ig = "http://localhost:5000/api/instagram"
url_tw = "http://localhost:5000/api/twitter"

users_data = []

for ix, id in enumerate(ids):
    user_data_dict = {}
    user_data_dict["id"] = id
    user_data_dict["name"] = names[ix]
    user_data_dict["instagram_username"] = instagram_usernames[ix]
    user_data_dict["twitter_username"] = twitter_usernames[ix]
    print(f"Working on {names[ix]}")

    if user_data_dict["instagram_username"] != None:
        
        print("Getting IG data")
        body = {"username": user_data_dict["instagram_username"]}
        r = requests.post(url_ig, json=body)
        response = r.json()
        user_data_dict["instagram_data"] = response

    if user_data_dict["twitter_username"] != None:
        print("Getting TW data")
        body = {"username": user_data_dict["twitter_username"]}
        r = requests.post(url_tw, json=body)
        response = r.json()
        user_data_dict["twitter_data"] = response
    
    users_data.append(user_data_dict)
    print(f"Done with {names[ix]}")
    print("----------------------------------")

# --- Process the data, compute engagement, percentiles and spi ---
def compute_spi_ig(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_posts_total": 0.1, 
        "n_likes_retrieved": 0.1, 
        "n_comments_retrieved": 0.1, 
        "relative_engagement": 0.2,
        "sentiment_instagram": 0.3
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_ig_data(data):
    users_data_instagram = []
    for user_data in data:
        if ("instagram_data" in user_data.keys()):
            dict_data = user_data["instagram_data"].copy()
            dict_data["user_id"] = user_data["id"]
            users_data_instagram.append(dict_data)
        else:
            empty_dict = {'user_id': users_data["id"],
                        'username': '',
                        "user_id_instagram": '',
                        'n_followers': 0,
                        'n_following': 0,
                        'n_posts_total': 0,
                        'n_posts_retrieved': 0,
                        'n_likes_total': 0,
                        'n_likes_retrieved': 0,
                        'n_comments_total': 0,
                        'n_comments_retrieved': 0,
                        'created_at': '',
                        'sentiment_instagram': 0}
            users_data_instagram.append(empty_dict)
        
    df_ig = pd.DataFrame(users_data_instagram)
    df_ig["relative_engagement"] = np.clip((df_ig["n_likes_retrieved"] + df_ig["n_comments_retrieved"])/(df_ig["n_followers"])*100, 0, 100)
    selected_cols = ["n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    df_ig_perc = df_ig.copy()
    for col in selected_cols:
        df_ig_perc[col] = df_ig_perc[col].apply(lambda x: stats.percentileofscore(df_ig_perc[col], x))
    df_ig_perc["spi"] = df_ig_perc[selected_cols].apply(compute_spi_ig, axis=1)
    spi_ig_columns = ["user_id", "username", "n_followers", "n_posts_total", "n_likes_retrieved", "n_comments_retrieved", "sentiment_instagram", "relative_engagement"]
    return pd.concat([df_ig[spi_ig_columns], df_ig_perc["spi"], df_ig["created_at"]], axis=1)

def compute_spi_tw(x):
    spi_ig_weights_dict = {
        "n_followers": 0.1,
        "n_retweets": 0.1, 
        "n_tweets": 0.1, 
        "n_retweets_to_user": 0.1, 
        "n_favorites_to_user": 0.1, 
        "n_replies_to_user": 0.1,
        "sentiment_twitter": 0.3,
        "relative_engagement": 0.1
    }
    spi = 0
    for k in spi_ig_weights_dict.keys():
        spi += spi_ig_weights_dict[k]*x[k]
    return spi

def process_tw_data(data):
    users_data_twitter = []
    for user_data in data:
        if ("twitter_data" in user_data.keys()):
            dict_data = user_data["twitter_data"].copy()
            dict_data["user_id"] = user_data["id"]
            users_data_twitter.append(dict_data)
        else:
            empty_dict = {'user_id': user_data["id"],
                        'username': '',
                        "n_followers": 0,
                        'n_retweets': 0,
                        'n_tweets': 0,
                        'n_retweets_to_user': 0,
                        'n_favorites_to_user': 0,
                        'n_replies_to_user': 0,
                        'created_at': '',
                        'sentiment_twitter': 0}
            users_data_twitter.append(empty_dict)
    
    df_tw = pd.DataFrame(users_data_twitter)
    df_tw["relative_engagement"] = (df_tw.apply(lambda x: (x["n_favorites_to_user"] + x["n_retweets_to_user"] + x["n_replies_to_user"])/(x["n_followers"])*100 if x["n_tweets"] > 0 else 0, axis=1))
    selected_cols = ["n_followers", "n_retweets", "n_tweets", "n_retweets_to_user", "n_favorites_to_user", "n_replies_to_user", "sentiment_twitter", "relative_engagement"]
    df_tw_perc = df_tw.copy()
    for col in selected_cols:
        df_tw_perc[col] = df_tw_perc[col].apply(lambda x: stats.percentileofscore(df_tw_perc[col], x))
    df_tw_perc["spi"] = df_tw_perc[selected_cols].apply(compute_spi_tw, axis=1)
    spi_tw_columns = ["user_id","username","n_followers","n_retweets","n_tweets","n_retweets_to_user","n_favorites_to_user","n_replies_to_user","sentiment_twitter","relative_engagement"]
    return pd.concat([df_tw[spi_tw_columns], df_tw_perc["spi"], df_tw["created_at"]], axis=1)

processed_ig_data = process_ig_data(users_data)
processed_tw_data = process_tw_data(users_data)

# --- Compute spi bonus ---

def input_ig_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO instagram_stats(user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["username"], record_dict["n_followers"], record_dict["n_posts_total"], record_dict["n_likes_retrieved"], record_dict["n_comments_retrieved"], record_dict["sentiment_instagram"], record_dict["relative_engagement"], record_dict["spi"]))

def retrieve_latest_record_ig(user_id):
    cur.execute(f"SELECT * FROM instagram_stats WHERE user_id = {user_id} ORDER BY created_at DESC LIMIT 1;") 
    for record_id, user_id, instagram_username, n_followers, n_posts_total, n_likes_retrieved, n_comments_retrieved, sentiment, engagement, spi, created_at in cur:
        record_dict = {
            "user_id": user_id,
            "username": instagram_username,
            "n_followers": n_followers,
            "n_posts_total": n_posts_total,
            "n_likes_retrieved": n_likes_retrieved,
            "n_comments_retrieved": n_comments_retrieved,
            "sentiment_instagram": sentiment,
            "relative_engagement": engagement,
            "spi": spi,
            "created_at": created_at
        }
    return pd.DataFrame(record_dict, index=[0])
      

def compute_definitive_ig_spi(user_id, new_record, bonus_weight=0.15):
    cur.execute(f"SELECT COUNT(*) FROM instagram_stats WHERE user_id = {user_id};") 
    for result in cur: 
        count_records = result[0]
    if count_records == 0:
        return new_record
    else: 
        
        dict_change = {}
        num_cols = [col for col in new_record.columns if is_numeric_dtype(new_record[col])]
        dict_change["username"] = new_record["username"].unique()[0]

        latest_record_db = retrieve_latest_record_ig(user_id)
        
        for col in num_cols:
            dict_change[col] = new_record.loc[0][col] - latest_record_db.loc[0][col]
            
        df_change = pd.DataFrame(dict_change, index=[0])
        
        dict_relative_change = {}
        dict_change = df_change.to_dict('records')[0]
        dict_base = latest_record_db.loc[0].to_dict()

        for key in dict_change.keys():
            if key == "username":
                dict_relative_change[key] = dict_change[key]
            elif key == "created_at" or key == "spi":
                pass
            else:
                if dict_base[key] == 0 and dict_change[key] != 0:
                    dict_relative_change[key] = 1
                elif dict_base[key] == 0 and dict_change[key] == 0:
                    dict_relative_change[key] = 0
                else:
                    dict_relative_change[key] = dict_change[key] / dict_base[key]
                    
        dict_ig_spi_bonus_weight = {
            'n_followers': 0.1,
            'n_posts_total': 0.1,
            'n_likes_retrieved': 0.1,
            'n_comments_retrieved': 0.1,
            'sentiment_instagram': 0.4,
            'relative_engagement': 0.2
        }

        bonus_spi = 0
        for k in dict_relative_change.keys():
            if k != "username" and k != "user_id":
                bonus_spi += dict_ig_spi_bonus_weight[k]*dict_relative_change[k]

        new_record_upd = new_record.copy()
        new_record_upd.loc[0, "spi"] = new_record_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100

        return new_record_upd[:1]

def input_tw_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO twitter_stats(user_id, twitter_username, n_followers, n_retweets , n_tweets , n_retweets_to_user , n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["username"], record_dict["n_followers"], record_dict["n_retweets"], record_dict["n_tweets"], record_dict["n_retweets_to_user"], record_dict["n_favorites_to_user"], record_dict["n_replies_to_user"], record_dict["sentiment_twitter"], record_dict["relative_engagement"], record_dict["spi"]))

def retrieve_latest_record_tw(user_id):
    cur.execute(f"SELECT * FROM twitter_stats WHERE user_id = {user_id} ORDER BY created_at DESC LIMIT 1;") 
    for record_id, user_id, twitter_username, n_followers, n_retweets, n_tweets, n_retweets_to_user, n_favorites_to_user, n_replies_to_user, sentiment, engagement, spi, created_at in cur:
        record_dict = {
            "user_id": user_id,
            "username": twitter_username,
            "n_followers": n_followers,
            "n_retweets": n_retweets,
            "n_tweets": n_tweets,
            "n_retweets_to_user": n_retweets_to_user,
            "n_favorites_to_user": n_favorites_to_user,
            "n_replies_to_user": n_replies_to_user,
            "sentiment_twitter": sentiment,
            "relative_engagement": engagement,
            "spi": spi,
            "created_at": created_at
        }
    return pd.DataFrame(record_dict, index=[0])
      

def compute_definitive_tw_spi(user_id, new_record, bonus_weight=0.15):
    cur.execute(f"SELECT COUNT(*) FROM twitter_stats WHERE user_id = {user_id};") 
    for result in cur: 
        count_records = result[0]
    if count_records == 0:
        return new_record
    else: 
        
        dict_change = {}
        num_cols = [col for col in new_record.columns if is_numeric_dtype(new_record[col])]
        dict_change["username"] = new_record["username"].unique()[0]

        latest_record_db = retrieve_latest_record_tw(user_id)
        for col in num_cols:
            dict_change[col] = new_record.loc[0][col] - latest_record_db.loc[0][col]
            
        df_change = pd.DataFrame(dict_change, index=[0])
        
        dict_relative_change = {}
        dict_change = df_change.to_dict('records')[0]
        dict_base = latest_record_db.loc[0].to_dict()

        for key in dict_change.keys():
            if key == "username":
                dict_relative_change[key] = dict_change[key]
            elif key == "created_at" or key == "spi":
                pass
            else:
                if dict_base[key] == 0 and dict_change[key] != 0:
                    dict_relative_change[key] = 1
                elif dict_base[key] == 0 and dict_change[key] == 0:
                    dict_relative_change[key] = 0
                else:
                    dict_relative_change[key] = dict_change[key] / dict_base[key]
                    
        dict_tw_spi_bonus_weight = {
        'n_followers': 0.1,
        'n_retweets': 0.1,
        'n_tweets': 0.1,
        'n_retweets_to_user': 0.1,
        'n_favorites_to_user': 0.1,
        'n_replies_to_user': 0.1,
        'sentiment_twitter': 0.3,
        'relative_engagement': 0.1
    }

        bonus_spi = 0
        for k in dict_relative_change.keys():
            if k != "username" and k != "user_id":
                bonus_spi += dict_tw_spi_bonus_weight[k]*dict_relative_change[k]

        new_record_upd = new_record.copy()
        new_record_upd.loc[0, "spi"] = new_record_upd.loc[0, "spi"] + bonus_weight*bonus_spi*100

        return new_record_upd[:1]


# --- Compute general SPI and input the data into the database ---

df_ig_fully_proc = pd.DataFrame()
for user_id in processed_ig_data["user_id"].unique():
    new_record = processed_ig_data[processed_ig_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    record_updated = compute_definitive_ig_spi(user_id, new_record)
    df_ig_fully_proc = pd.concat([df_ig_fully_proc, record_updated], axis=0, ignore_index=True)
    input_ig_data(cur, record_updated)

df_tw_fully_proc = pd.DataFrame()
for user_id in processed_tw_data["user_id"].unique():
    new_record = processed_tw_data[processed_tw_data["user_id"] == user_id].reset_index().drop("index", axis=1)
    record_updated = compute_definitive_tw_spi(user_id, new_record)
    df_tw_fully_proc = pd.concat([df_tw_fully_proc, record_updated], axis=0, ignore_index=True)
    input_tw_data(cur, record_updated)

def input_spi_data(cur, record):
    record_dict = record.to_dict('records')[0]
    cur.execute("INSERT INTO spis(user_id, spi, spi_instagram, spi_twitter, created_at) VALUES (?, ?, ?, ?, now());",
      (record_dict["user_id"], record_dict["spi"], record_dict["spi_instagram"], record_dict["spi_twitter"]))

spi_df = pd.merge(df_ig_fully_proc[["user_id", "spi"]], df_tw_fully_proc[["user_id", "spi"]], on="user_id", suffixes=["_instagram", "_twitter"])
weight_ig = 0.7
weight_tw = 0.3
spi_df["spi"] = spi_df["spi_instagram"] * weight_ig +  spi_df["spi_twitter"] * weight_tw

for user_id in spi_df["user_id"].unique():
    new_record = spi_df[spi_df["user_id"] == user_id].reset_index().drop("index", axis=1)
    input_spi_data(cur, new_record) 

conn.commit()
conn.close()
